In [1]:
import shapefile
from shapely.geometry import shape, Point, mapping
import numpy as np
import pandas as pd

base = "../../data/raw/"
clean = "../../data/processed/"
filename = "2016_LGA_SHAPE/LGA_2016_AUST"

sf = shapefile.Reader(base+filename)

nswshapes = list(map(lambda x: shape(x.__geo_interface__), sf.shapes()[:130]))
centroids = list(map(lambda x: shape(x.__geo_interface__).centroid, sf.shapes()[:130]))

In [2]:
import pandas as pd
df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "unknown2"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()
nswdf



,code,codenum,name,unknown,state,unknown2,clean_name
0,LGA10050,10050,Albury (C),1,New South Wales,305.9459,Albury
1,LGA10130,10130,Armidale Regional (A),1,New South Wales,8620.6990,Armidale Regional
2,LGA10250,10250,Ballina (A),1,New South Wales,484.9389,Ballina
3,LGA10300,10300,Balranald (A),1,New South Wales,21690.6753,Balranald
4,LGA10470,10470,Bathurst Regional (A),1,New South Wales,3817.8646,Bathurst Regional
5,LGA10550,10550,Bega Valley (A),1,New South Wales,6278.8811,Bega Valley
6,LGA10600,10600,Bellingen (A),1,New South Wales,1600.4337,Bellingen
7,LGA10650,10650,Berrigan (A),1,New South Wales,2065.7759,Berrigan
8,LGA10750,10750,Blacktown (C),1,New South Wales,240.0487,Blacktown
9,LGA10800,10800,Bland (A),1,New South Wales,8557.6535,Bland


In [85]:
import json
lga_json = list(map(lambda x: x.__geo_interface__, sf.shapes()[:130]))

geojson = {'type': 'FeatureCollection',
            'features': [{'type':'Feature', 'geometry':x, 'properties': {'name':nswdf.iloc[i]['clean_name']}} for i, x in enumerate(lga_json)]
          }


#with open('geojson.json','w') as f:
#    json.dump(geojson, f)
#geojson[:5]

In [86]:
#target = pd.read_csv(base+"cleaned_target.csv")

#set(nswdf["clean_name"]).difference(set(target["LGA"]))
#set(target["LGA"]).difference(set(nswdf["clean_name"]))

In [20]:
import os

tweetdf =pd.read_csv(os.path.join(clean, "opal_preshapes.csv"),header=0)



/Users/tdm/git/capstone/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
loctweetdf = tweetdf.loc[pd.isnull(tweetdf['lga'])]

In [5]:
len(loctweetdf)

104636

In [90]:
#loctweetdf = loctweetdf[loctweetdf["location_name"] != "New South Wales"]

In [6]:
def whichlga(tweetpoints, lgashapes):
    #loop over 130 NSW LGAs and return a name or "None"
    #NB you get names from Census data which is 2016 - pre amalgamations
    #test by whether in bounding box. If in two, go to shapes. If still in two, one with smaller bounding box
    output = []
    nums = []
    amalgamations = {'Botany Bay': 'Bayside',
                     'Rockdale'  : 'Bayside',
                     'Gundagai'  : 'Cootamundra-Gundagai',
                     'Western Plains Regional' : 'Dubbo Regional',
                     'Unincorporated NSW' : 'Unincorporated Far West'
                    }
    
    for point in tweetpoints:
        found = []
        distances = []
        for i, nswshape in enumerate(nswshapes):
            if point.within(nswshape):
                #import pdb; pdb.set_trace()
                found.append(i)
                break
            else:
                distances.append(point.distance(nswshape))
                
        #print(len(found))
        nums.append(len(found))
        if len(found) == 1:
            clean_name = nswdf.iloc[found[0]].clean_name
        else:
            #out on the water
            clean_name = nswdf.iloc[np.argmin(np.asarray(distances))].clean_name
            
        if clean_name in amalgamations:
            clean_name = amalgamations[clean_name]
        output.append(clean_name)
            
    return pd.Series(output), nums
            

In [7]:
tweetpoints = []

loctweetdf.apply(lambda x: tweetpoints.append(shape(Point(x['lng'], x['lat']))), axis=1)


0         None
1         None
3         None
5         None
6         None
9         None
10        None
11        None
12        None
13        None
17        None
20        None
27        None
28        None
32        None
36        None
37        None
38        None
42        None
48        None
52        None
55        None
58        None
59        None
60        None
63        None
64        None
69        None
70        None
74        None
          ... 
407900    None
407902    None
407903    None
407905    None
407906    None
407908    None
407909    None
407911    None
407914    None
407916    None
407918    None
407919    None
407921    None
407923    None
407925    None
407927    None
407929    None
407931    None
407934    None
407935    None
407938    None
407939    None
407941    None
407942    None
407944    None
407945    None
407946    None
407948    None
407950    None
407952    None
Length: 104636, dtype: object

In [25]:

output, found = whichlga(tweetpoints, nswshapes)
found = np.asarray(found)
np.histogram(found)

KeyboardInterrupt: 

In [22]:
loctweetdf[found == 0]

,Unnamed: 0,mode,date,tap,time,loc,count,lga,state,lat,lng
18241,18734,bus,20160726,on,15:15,2319,27,NaN,NSW,-32.704315,152.004814
18780,19287,bus,20160726,on,07:15,2319,19,NaN,NSW,-32.704315,152.004814
19946,20482,bus,20160726,on,07:00,2319,19,NaN,NSW,-32.704315,152.004814
20252,20796,bus,20160726,on,08:45,2319,20,NaN,NSW,-32.704315,152.004814
20997,21560,bus,20160726,on,07:30,2319,51,NaN,NSW,-32.704315,152.004814
24493,25140,bus,20160726,on,07:45,2319,41,NaN,NSW,-32.704315,152.004814
24901,25557,bus,20160726,on,08:15,2319,18,NaN,NSW,-32.704315,152.004814
26664,27361,bus,20160726,off,15:15,2319,31,NaN,NSW,-32.704315,152.004814
27376,28089,bus,20160726,off,09:00,2319,20,NaN,NSW,-32.704315,152.004814
29685,30459,bus,20160726,off,16:00,2319,39,NaN,NSW,-32.704315,152.004814


In [26]:

tweetdf.loc[tweetdf['lga'].isnull(),'lga'] = np.asarray(output)
#tweetdf.loc[(tweetdf["location_name"] == "Glenn Innis Severn"), "lga"] =  "Glen Innes Severn"

In [50]:
#time of day
tweetdf["hour"] = tweetdf["time"].str.slice(stop=2).astype(np.int)
tweetdf["timeofday"] = tweetdf["hour"] //6
tod_tweetdf = tweetdf.drop("timeofday", axis=1).join(pd.get_dummies(tweetdf["timeofday"]))
tod_tweetdf = tod_tweetdf.rename(columns = {0:"night", 1:"morning", 2:"afternoon", 3:"evening"})
tod_tweetdf

,Unnamed: 0,mode,date,tap,time,loc,count,lga,state,lat,lng,hour,night,morning,afternoon,evening
0,0,bus,20160730,on,02:30,2000,415,Sydney,NSW,-33.870846,151.207330,2,1,0,0,0
1,1,bus,20160730,on,02:30,2135,18,Strathfield,NSW,-33.880881,151.076250,2,1,0,0,0
2,3,bus,20160730,on,02:30,2010,31,Sydney,NSW,-33.882559,151.213158,2,1,0,0,0
3,4,bus,20160730,on,13:30,2095,64,Northern Beaches,NSW,-33.806016,151.294777,13,0,0,1,0
4,5,bus,20160730,on,13:30,2097,25,Northern Beaches,NSW,-33.734283,151.294777,13,0,0,1,0
5,6,bus,20160730,on,13:30,2220,112,Georges River,NSW,-33.958369,151.099545,13,0,0,1,0
6,7,bus,20160730,on,13:30,2093,28,Northern Beaches,NSW,-33.805574,151.262708,13,0,0,1,0
7,8,bus,20160730,on,13:30,2099,72,Northern Beaches,NSW,-33.750008,151.286031,13,0,0,1,0
8,9,bus,20160730,on,13:30,2529,26,Shellharbour,NSW,-34.597194,150.855121,13,0,0,1,0
9,10,bus,20160730,on,13:30,2528,33,Shellharbour,NSW,-34.556663,150.855121,13,0,0,1,0


In [51]:
import csv
tod_tweetdf.to_csv(clean+"opal_lgas.csv", quoting=csv.QUOTE_NONNUMERIC)

In [24]:
len(tweetdf.groupby("lga").count())

47